In [150]:
from pdf2image import convert_from_path
import os
import json

In [151]:
functions = [
	{
		"q": {
			"text": "Simplifica",
			"math": "15+23-35"
		     },
		"correct": "3",
		"incorrect": ["4", "5", "2"]
	},
	{
		"q": {
			"text": "Simplifica",
			"math": "(2x4+12)(6-4)"
		     },
		"correct": "40",
		"incorrect": ["60", "32", "52"]
	},
	{
		"q": {
			"text": "Simplifica",
			"math": "3x9+(6+5x3)-\\frac{12}{4}"
		},
		"correct": "60",
		"incorrect": ["72", "48", "54"]
	},
	{
		"q": {
			"text": "Simplifica",
			"math": "\\frac{3x4+5}{2x3+1}"
		},
		"correct": "\\frac{17}{7}",
		"incorrect": ["\\frac{17}{8}", "\\frac{17}{6}", "\\frac{17}{9}"]
	},
	{
		"q": {
			"text": "Deriva",
			"math": "7x^2 + 2x"
		},
		"correct": "14x + 2",
		"incorrect": ["2x + 14", "21x + 2", "14x + x"]
	},
	{
		"q": {
			"text": "Deriva",
			"math": "\\frac{x^2}{x+1}"
		},
		"correct": "\\frac{x(x+2)}{(x+1)^2}",
		"incorrect": ["\\frac{x(x+1)}{(x+2)^2}", "2x", "\\frac{x+1}{2x}"]
	}
]

In [152]:
def parse_latex(functions):
    levels = []
    for level, function in enumerate(functions):
        level_dict = {
            "question": {
                "type": "Image",
                "value": f"{level}/question.png"
            },
            "answers": [{
                "type": "Image",
                "value": f"{level}/correct.png",
                "answer": True
            }]
        }
        # Question
        qtex = f"""
                \\documentclass{{slides}}
                \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
                \\begin{{document}}
                \\begin{{Large}}
                \\begin{{center}}
                    {function["q"]["text"]}
                    \\[ {function["q"]["math"]} \\]
                \\end{{center}}
                \\end{{Large}}
                \\end{{document}}
                """
        with open("qtemp.tex", "w") as f:
            f.write(qtex)
            f.close()
        os.system("pdflatex qtemp.tex")
        qimage = convert_from_path("qtemp.pdf")
        os.mkdir(f"./game/assets/questions/{level}")
        qimage[0].save(f"./game/assets/questions/{level}/question.png", "PNG")

        # correct answer
        tex = f"""
              \\documentclass{{slides}}
              \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
              \\begin{{document}}
              \\begin{{huge}}
                \\[ {function['correct']} \\]
              \\end{{huge}}
              \\end{{document}}
              """
        with open("ctemp.tex", "w") as f:
            f.write(tex)
            f.close()
        os.system("pdflatex ctemp.tex")
        cimage = convert_from_path("ctemp.pdf")
        cimage[0].save(f"./game/assets/questions/{level}/correct.png", "PNG")

        # incorrect answers
        for i, answer in enumerate(function['incorrect']):
            incorrect_answer = {
                "type": "Image",
                "value": f"{level}/incorrect{i}.png",
                "answer": False
            }
            level_dict["answers"].append(incorrect_answer)
            tex = f"""
                  \\documentclass{{slides}}
                  \\usepackage[paperheight=4.17in,paperwidth=6.25in]{{geometry}}
                  \\begin{{document}}
                  \\begin{{huge}}
                    \\[ {answer} \\]
                  \\end{{huge}}
                  \\end{{document}}
                  """
            with open("temp.tex", "w") as f:
                f.write(tex)
                f.close()
            os.system("pdflatex temp.tex")
            image = convert_from_path("temp.pdf")
            image[0].save(f"./game/assets/questions/{level}/incorrect{i}.png", "PNG")
        
        levels.append(level_dict)
    open("game/assets/questions/questions.json", "w").write(json.dumps(levels))


In [153]:
parse_latex(functions)